<a href="https://colab.research.google.com/github/david3080/llmapi/blob/main/1_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. チャットアプリ開発

Streamlitを使用した、OpenAIのGPT-4モデルとチャットするWebアプリケーションの作成

### ステップ1. 必要なライブラリのインストール
streamlitとopenaiのライブラリをインストールします。

In [ ]:
%pip install streamlit openai

### ステップ2. Streamlitのアプリケーションを作成する
Streamlitのアプリケーションを作成するためのファイルを作成します。
このファイルは、app.pyという名前で保存します。

In [ ]:
%%writefile app.py
import openai
import streamlit as st

# セッション状態にall_textがない場合は、空のリストを作成する
if "all_text" not in st.session_state:
    st.session_state.all_text = []

# サイドバーにAPIキーを入力するフォームを作成する
with st.sidebar:
    st.title("AIチャット")
    api_key = st.text_input("OPEN_AI_KEY", type="password")

if api_key:
    # APIキーが入力された場合は、openaiのAPIキーを設定する
    openai.api_key = api_key
    # ユーザープロンプトの入力エリアを取得する
    user_prompt = st.chat_input("user:")
    # アシスタントのテキストを空にする
    assistant_text = ""
    # ユーザーが入力したテキストとアシスタントが返したテキストの履歴を表示する
    for text_info in st.session_state.all_text:
        with st.chat_message(text_info["role"], avatar=text_info["role"]):
            st.write(text_info["role"] + ":\n\n" + text_info["content"])

    # ユーザによるプロンプトがある場合は、ユーザのプロンプトを追加する
    if user_prompt:
        with st.chat_message("user", avatar="user"):
            st.write("user" + ":\n\n" + user_prompt)

        st.session_state.all_text.append({"role": "user", "content": user_prompt})

        # ユーザーのプロンプトが10個以上ある場合は、最初のプロンプトを削除する
        if len(st.session_state.all_text) > 10:
            st.session_state.all_text.pop(1)

        # ユーザーのプロンプトをGPT-4モデルで応答する
        response = openai.chat.completions.create(
            model="gpt-4",
            messages=st.session_state.all_text,
            stream=True,
        )
        with st.chat_message("assistant", avatar="assistant"):
            place = st.empty()
            for chunk in response:
                content = chunk.choices[0].delta.content
                if content:
                    assistant_text += content
                    place.write("assistant" + ":\n\n" + assistant_text)

        st.session_state.all_text.append(
            {"role": "assistant", "content": assistant_text}
        )
else:
    # ユーザがAPIキーを入力していない場合は、入力を促すメッセージを表示する
    st.info("👈OPEN_AI_KEYを入力してください。")

: 

In [ ]:
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501